In [ ]:
import pandas as pd
import nltk
import re

Loading Dataset

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/IBM/spam.csv',encoding="ISO-8859-1")

**Data Preprocessing**

Encoding the categorical column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
data['v1']=le.fit_transform(data['v1'])

In [ ]:
data.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,1,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,0,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,0,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,1,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,1,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
pe=PorterStemmer()
arr=[]

In [ ]:
data.shape
(5572,5)

(5572, 5)

In [ ]:
for i in range(5572):
  msg=data['v2'][i]
  msg=re.sub('[^a-zA-Z]',' ',msg)
  msg=msg.lower()
  msg=msg.split()
  msg=[pe.stem(word)for word in msg if not word in set(stopwords.words('english'))]
  msg=' '.join(msg)
  arr.append(msg)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000)

In [26]:
x=cv.fit_transform(arr).toarray()

In [28]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
x.shape

(5572, 6221)

In [29]:
y=data['v1'].values

In [30]:
y

array([0, 0, 1, ..., 0, 0, 0])

Train and Test split

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

model building

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [44]:
model=Sequential()

In [45]:
model.add(Dense(units=6221,activation='relu'))
model.add(Dense(units=8000,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [46]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [47]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
140/140 [==============================] - 107s 752ms/step - loss: 0.1119 - accuracy: 0.9717
Epoch 2/5
140/140 [==============================] - 115s 823ms/step - loss: 0.0079 - accuracy: 0.9980
Epoch 3/5
140/140 [==============================] - 125s 897ms/step - loss: 7.5131e-04 - accuracy: 0.9998
Epoch 4/5
140/140 [==============================] - 105s 744ms/step - loss: 1.7114e-04 - accuracy: 1.0000
Epoch 5/5
140/140 [==============================] - 113s 805ms/step - loss: 5.5876e-05 - accuracy: 1.0000


saving model

In [48]:
ls

drive/  sample_data/


In [49]:
cd /content/drive/MyDrive/IBM

/content/drive/MyDrive/IBM


In [50]:
model.save('spamdetector.h5')

testing model

In [51]:
from tensorflow.keras.models import load_model

In [52]:
model=load_model('/content/drive/MyDrive/IBM/spamdetector.h5')

In [53]:
def preprocess(msg):
  msg=re.sub('[^a-zA-Z]',' ',msg)
  msg=msg.lower()
  msg=msg.split()
  msg=[pe.stem(word) for word in msg if not word in set(stopwords.words('english'))]
  msg=' '.join(msg)
  return msg

In [55]:
message='Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&Cs apply 08452810075over18s'

In [56]:
message=preprocess(message)

In [57]:
message

'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate cs appli'

In [59]:
pred=model.predict(cv.transform([message]))

1/1 [==============================] - 0s 262ms/step


In [60]:
pred

array([[1.]], dtype=float32)

In [61]:
if(pred>0.5):
  print("Spam Message!!!!!")
else:
  print("Not Spam")

Spam Message!!!!!
